In [1]:
import numpy as np
import pandas as pd

In [2]:

DATAPATH = "/home/miki/Desktop/Deployment/bacillus-anthracis/data/raw/train.csv"
df = pd.read_csv(DATAPATH)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    48 non-null     object
 1   case    48 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 896.0+ bytes


In [4]:
df

,date,case
0,2014-01-31,132
1,2014-02-28,175
2,2014-03-31,240
3,2014-04-30,314
4,2014-05-31,299
5,2014-06-30,156
6,2014-07-31,253
7,2014-08-31,119
8,2014-09-30,188
9,2014-10-31,267


In [5]:
def makeXy(ts, nb_timesteps):
    """
    Input: 
           ts: original time series
           nb_timesteps: number of time steps in the regressors
    Output: 
           X: 2-D array of regressors
           y: 1-D array of target 
    """
    X = []
    y = []
    for i in range(nb_timesteps, ts.shape[0]):
        X.append(list(ts.loc[i-nb_timesteps:i-1]))
        y.append(ts.loc[i])
    X, y = np.array(X), np.array(y)
    return X, y

In [6]:
# Xtrain, ytrain = makeXy(df['case'], 12)

In [7]:
X_train, y_train = makeXy(df['case'], 11)
print('Shape of train arrays:', X_train.shape, y_train.shape)

Shape of train arrays: (37, 11) (37,)


In [8]:
df_val = pd.read_csv("/home/miki/Desktop/Deployment/bacillus-anthracis/data/raw/test.csv")

In [9]:
df_val

,date,case
0,2018-01-31,118
1,2018-02-28,97
2,2018-03-31,61
3,2018-04-30,34
4,2018-05-31,262
5,2018-06-30,68
6,2018-07-31,85
7,2018-08-31,60
8,2018-09-30,11
9,2018-10-31,3


In [10]:
X_val, y_val = makeXy(df_val['case'], 11)
print('Shape of validation arrays:', X_val.shape, y_val.shape)

Shape of validation arrays: (1, 11) (1,)


In [11]:
X_train, X_val = X_train.reshape((X_train.shape[0], X_train.shape[1], 1)), X_val.reshape((X_val.shape[0], X_val.shape[1], 1))
print('Shape of arrays after reshaping:', X_train.shape, X_val.shape)

Shape of arrays after reshaping: (37, 11, 1) (1, 11, 1)


In [13]:
import tensorflow as tf
from keras.models import *
from keras.callbacks import *
from keras.optimizers import *
from keras.layers import Dense, LSTM, Dropout
import random 

In [14]:
def set_seed(seed):
    
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed);


In [15]:
def get_model(X_train, y_train):
    
    set_seed(33)
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(11, 1)))
    model.add(Dense(1))
    model.add(Dropout(rate=0.1))
    model.compile(optimizer='adam', loss='mean_absolute_error')
    model.fit(X_train,y_train,epochs=100,batch_size=25,verbose=0)
    return model

In [16]:
mm = get_model(X_train, y_train);

2022-06-21 20:11:12.087928: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-21 20:11:12.088099: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-21 20:11:12.088174: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (miki): /proc/driver/nvidia/version does not exist
2022-06-21 20:11:12.111401: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
mm

In [18]:
mm.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                10400     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
 dropout (Dropout)           (None, 1)                 0         
                                                                 
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


In [19]:
MODELPATH  = '/home/miki/Desktop/Deployment/bacillus-anthracis/model'

In [22]:
save_weights_at = os.path.join(MODELPATH, 'Bacillus.{epoch:02d}-{val_loss:.4f}.hdf5')
save_best = ModelCheckpoint(save_weights_at, monitor='val_loss', verbose=0,
                            save_best_only=True, save_weights_only=False, mode='min',
                            period=1)
mm.fit(x=X_train, y=y_train,callbacks=[save_best], validation_data=(X_val, y_val), shuffle=True)

2/2 [==============================] - 0s 78ms/step - loss: 63.1352 - val_loss: 29.1740


In [24]:
from sklearn.preprocessing import RobustScaler, MinMaxScaler
scaler = RobustScaler() 

In [26]:
best_model = load_model(os.path.join('/home/miki/Desktop/Deployment/bacillus-anthracis/model/Bacillus.01-29.1740.hdf5'))
preds = best_model.predict(X_val)
pred_pm25 = (preds)
pred_pm25 = np.squeeze(pred_pm25)

1/1 [==============================] - 0s 129ms/step


In [33]:
pred_pm25

array(35.173954, dtype=float32)

In [35]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(df_val['case'].loc[1:], pred_pm25)
print('MAE for the validation set:', round(mae, 4))

TypeError: Singleton array array(35.173954, dtype=float32) cannot be considered a valid collection.